In [ ]:
-- Complex SQL Query 1 | Derive Points table for ICC tournament

/*select
    team_name, count(1) as no_of_matches_play,
    sum(win_flag) as no_of_matches_won,
    count(1) - sum(win_flag) as no_of_matches_loss
FROM
(
    select
        team_1 as team_name, case when team_1 = winner then 1 else 0 end as win_flag
    from icc_world_cup
    UNION ALL
    select
        team_2 as team_name, case when team_2 = winner then 1 else 0 end as win_flag
    from icc_world_cup
) t
group by team_name
order by no_of_matches_won desc;*/


WITH icc_match_info as
(
    select team_1 as team_name, winner from icc_world_cup
    union all
    select team_2 as team_name, winner from icc_world_cup
)

select 
    team_name,
    count(*) as total_matches_played,
    sum(case when team_name = winner then 1 else 0 end) as no_of_wins,
    sum(case when team_name != winner then 1 else 0 end) as no_of_loss
from icc_match_info
group by team_name
order by no_of_wins desc, team_name

In [ ]:
-- Complex SQL 2 | find new and repeat customers | SQL Interview Questions
-- order_date, new_customer_count, repeat_customer_count


WITH first_visit as
(
    select
        customer_id,
        min(order_date) as first_visit_date
    from customer_orders
    group by customer_id
)
select
    co.order_date,
    sum(case when order_date = first_visit_date then 1 else 0 end) as new_cust_count,
    sum(case when order_date != first_visit_date then 1 else 0 end) as repeat_cust_count
from 
    customer_orders co join first_visit fv
on 
    co.customer_id = fv.customer_id
group by 
    co.order_date

-- I was asked this same question today in amazon interview. The question was lil changed but concept was same. They needed new user counts in a year 2022 only and wanted the output to be displayed as weekly level. So for week 1 how many new users till week 52. I was able to solve this though i took some time not sure if they judge on the speed as well. Thanks for this video

In [ ]:
with 
total_visits as (
  select name, count(1) as total_visits, LISTAGG(DISTINCT resources, ', ') WITHIN GROUP (ORDER BY resources) AS resources_used,
  from entries 
  group by name 
),
floor_visit as (
  select 
    name, floor, count(1) no_of_floor_visit, 
    rank() over(partition by name order by count(1) desc) as rn 
  from entries 
  group by name, floor
)
select 
    fv.name, 
    tv.total_visits, 
    fv.floor as most_visited_floor,
    tv.resources_used
from floor_visit fv
inner join total_visits tv on fv.name = tv.name
where rn = 1
order by name